## Skup za validaciju

Umesto podele na trening i test skup, polazni skup podataka delimo na tri skupa:
1. skup za treniranje
2. skup za validaciju
3. skup za testiranje
<img src='validation.png'>

Opisanu podelu skupa podataka dobijamo pozivanjem `train_test_split` funkcije iz dva puta.  

In [ ]:
%run kNN.ipynb

In [ ]:
# prvo izdvajamo test skup kao 30% svih podataka
X_train_validation, X_test, y_train_validation, y_test = model_selection.train_test_split(X, y, test_size=0.3, stratify = y, random_state = 25)

In [ ]:
# zatim izdvajamo validacioni skup kao 20% prvobitnog trening skupa
X_train, X_validation, y_train, y_validation = model_selection.train_test_split(X_train_validation, y_train_validation, test_size = 0.2, stratify = y_train_validation, random_state = 25)

Trazimo optimalni hiperparametar, u ovom slucaju k kao broj suseda za kNN algoritam. Pravimo niz `ks` koji sadrzi vrednosti parametara koje cemo ocenjivati na validacionom skupu.

In [ ]:
ks = np.arange(20) + 1
ks

### Bitno: I pri trazenju optimalnih parametara, kao i do sada pri testiranju, skaliranje radimo samo na osnovu podataka iz trening skupa! Validacioni skup mora biti nepristrasan zbog ocenjivanja parametara.

In [ ]:
scaler_tv = preprocessing.StandardScaler()
scaler_tv.fit(X_train) # samo trening skup se fituje
X_train = scaler_tv.transform(X_train)
X_validation = scaler_tv.transform(X_validation)

Dakle treniramo model sa svim mogucim kombinacijama datih nizova parametara (nekada ih je vise) i trazimo optimalnu kao onu za koju je model postigao najbolju ocenu na validacionom skupu:

In [ ]:
# inicijalizacija najboljeg skora i najboljeg parametra
best_score = 0 
best_params = {'k':0}

In [ ]:
for k in ks: 
    model = neighbors.KNeighborsClassifier(n_neighbors=k) 
    model.fit(X_train, y_train)

    y_pred = model.predict(X_validation)
    score = metrics.accuracy_score(y_validation, y_pred)

    if score > best_score: # ako smo dobili bolje resenje od postojeceg menjamo ga
        best_score = score
        best_params['k'] = k

In [ ]:
best_score

In [ ]:
best_params

### Kada izaberemo optimalne vrednosti hiperparametara modela, treniranje vrsimo na celom prvobitnom trening skupu (trening + validacija), a test skup koristimo samo za evaluaciju izabranog modela!

In [ ]:
# skaliramo kao i do sada
scaler_tt = preprocessing.StandardScaler()
scaler_tt.fit(X_train_validation)
X_train_validation = scaler_tt.transform(X_train_validation)
X_test = scaler_tt.transform(X_test)

In [ ]:
best_params['k']

In [ ]:
model_with_best_params = neighbors.KNeighborsClassifier(n_neighbors=best_params['k'])

In [ ]:
model_with_best_params.fit(X_train_validation, y_train_validation)

In [ ]:
y_pred = model_with_best_params.predict(X_test)

In [ ]:
metrics.accuracy_score(y_test, y_pred)

## Cross validation

Opet se suocavamo sa istim problemom - rezultati, kako mera kvaliteta tako i izbora parametara, zavise od nasumicne podele podataka na skupove za trening, validaciju i test.

### Kada znamo optimalne parametre:

In [ ]:
model_with_best_params = neighbors.KNeighborsClassifier(n_neighbors=best_params['k'])

Radimo k-Fold validaciju na isti nacin kao i u skripti *KFoldValidation*:

In [ ]:
scaler = preprocessing.StandardScaler()

In [ ]:
from sklearn import pipeline
knn_pipeline =  pipeline.make_pipeline(scaler, model_with_best_params)

In [ ]:
scores = model_selection.cross_val_score(knn_pipeline, X, y, scoring='accuracy', cv = 10) # 10 slojeva

In [ ]:
scores

In [ ]:
scores.mean()

### Kada biramo optimalne parametre:

Koristimo klasu `GridSearchCV` kojoj prosledjujemo:
* Model za koji treba odrediti vrednosti parametara (odnosno ceo pipeline modela)
* Sve parametare koje treba razmotriti (dictionary struktura)
* Funkciju za ocenu modela
* Broj slojeva 

In [ ]:
grid_model = pipeline.Pipeline(steps=[('scaler', preprocessing.StandardScaler()), 
                                      ('knn', neighbors.KNeighborsClassifier())])
params = {
    'knn__n_neighbors': np.arange(20) + 1   # key oblika model__parametar
}

In [ ]:
grid = model_selection.GridSearchCV(grid_model, param_grid=params, scoring='accuracy', cv=10)

<img src='kfold_validation.png'>

### Bitno: k-Fold validaciju za izbor parametara radimo za ceo trening + validacioni skup!

In [ ]:
grid.fit(X_train_validation, y_train_validation)

`GridSearchCV` ima svojstva `best_score_` i `best_params_` koja citamo:

In [ ]:
grid.best_score_

In [ ]:
grid.best_params_  # dobili smo novu optimalnu vrednost broja suseda k

Informacije o svakom koraku unakrsne validacije za svaku kombinaciju vrednosti hiperparametara sadrzane su u `cv_results_`:

In [ ]:
cv_results = pd.DataFrame(grid.cv_results_)

In [ ]:
cv_results.shape # 20 kombinacija/vrednosti hiperparametra i 19 svojstava za svaku od njih

In [ ]:
cv_results

In [ ]:
cv_results.columns

In [ ]:
cv_results['mean_test_score'] 
# ocena, odnosno tacnost, za svaku vrednost parametra k kao prosek svih ocena dobijenih na k-slojeva unakrsne validacije

In [ ]:
plt.plot(ks, cv_results['mean_test_score'])
plt.xlabel('Parametar k')
plt.ylabel('Tacnost')
plt.show()

## Finalni model i evaluacija

Nakon izbora najboljih parametara unakrnom validacijom, kao meru kvaliteta modela prijavljujemo njegovu ocenu na test skupu. Zato ga prvo treniramo na ostatku svih podataka koristeci izabrane parametre:

In [ ]:
model_best_grid = neighbors.KNeighborsClassifier(n_neighbors=grid.best_params_['knn__n_neighbors'])

In [ ]:
model_best_grid.fit(X_train_validation, y_train_validation)

In [ ]:
y_pred = model_best_grid.predict(X_test)

In [ ]:
metrics.accuracy_score(y_test, y_pred) # ovo je vrednost koju prijavljujemo kao meru kvaliteta modela

Ako imamo veliki broj podataka, ovde zavrsavamo, ali ako to nije slucaj i ne zelimo da gubimo informacije, model cemo jos jednom trenirati na celom skupu podataka i to sacuvati kao nas finalni model:

In [ ]:
final_scaler = preprocessing.StandardScaler()
X = final_scaler.fit_transform(X)

In [ ]:
final_model = neighbors.KNeighborsClassifier(n_neighbors=grid.best_params_['knn__n_neighbors'])

In [ ]:
final_model.fit(X, y)